In [1]:
from Bio import SeqIO
import numpy as np
from matplotlib import pyplot as plt
from progressbar import progressbar

In [2]:
gbk_file = "./YRp48lacO6(TATO7BHIS3MS2lacO6).gbk"

with open(gbk_file, "r") as handle:
    record = SeqIO.read(handle, "genbank")
    print(f"Record ID: {record.id}")
    print(f"Description: {record.description}")
    print(f"Sequence length: {len(record.seq)}")

Record ID: .
Description: synthetic circular DNA
Sequence length: 3070


In [3]:
def double_check_resembles(threshold=5):
    target = "ATGACTCAT"
    matches = {}
    for start in range(0,len(full_seq)-9):
        subseq = full_seq[start:start+9]
        nmatch = sum(1 for a, b in zip(subseq, target) if a == b)
        if nmatch >= threshold:
            matches[start] = nmatch
    return matches

def get_matches_fromZelin():
    target = "ATGACTCAT"
    matches = {} # start site: matches
    endSites = {} # start site: end sites
    for feature in progressbar(record.features):
        if feature.type == 'misc_feature':
            start = int(feature.location.start)
            end = int(feature.location.end)
            seq_str = str(feature.extract(record.seq))
            # Compare base by base
            matches[start] = sum(1 for a, b in zip(seq_str, target) if a == b)
            endSites[start] = end
    return matches, endSites

In [4]:
# Full sequence
full_seq = str(record.seq)
matches_doubleCheck = double_check_resembles()
prev_start = -100
for start in list(matches_doubleCheck.keys()):
    if start < prev_start + 9:
        if matches_doubleCheck[start] > matches_doubleCheck[prev_start]:
            matches_doubleCheck.pop(prev_start)
            prev_start = start
        else:
            matches_doubleCheck.pop(start)
startSites_doubleCheck = list(matches_doubleCheck.keys())
len(matches_doubleCheck)

156

In [5]:
matches, endSites = get_matches_fromZelin()
startSites = list(matches.keys())
for excluded in [980, 1146, 1893]: # 980 is the overlap, the other two are not
    if excluded in startSites: 
        startSites.remove(excluded)
        matches.pop(excluded)
        endSites.pop(excluded)
print("Minimum match:",min([matches[k] for k in matches]))
len(startSites)

  0% (0 of 96) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--
100% (96 of 96) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


Minimum match: 4


44

Notes: 
1. The start site given by `feature.location.start` is the python position starting from 0.
2. The end site given by `feature.location.end` can be understood as slicing `[start:end]`.
   
Problems:
1. 要处理重复的site，如果考虑全部的9个bases，其中977-985以及981-989的两个长度为9的片段会重复。
    - 其中第一段重复了7个，第二段重复了6个，按照一个site算，有7个重复的sites

In addition, the UAS site is at 937 - 945 and it has 8 sites resembling the strongest binding site `ATGACTCAT`.

In [6]:
UAS = {'start':936, 'end':945, 'matches':8}
HIS3 = {'start':884, 'end':1953}

Since all DNA oligomers contain the UAS site (or mutated UAS site), the start site of UAS is set to be at 0 in our model. Then, the relative positions of each site can be defined using this coordinate. Firstly, we want to compare our result with occupancy measures for san check.
- This is for $\Delta$UAS mutations
- For qualatitive comparison with G_SELEX, the following problems are considered:

We care about the sequence between TRP1 and the 6 lac operators, it has 2265 base pairs, which is 755 nm. This is much longer than the persistence length. 

Also, how to deal with nucleosomes?
1. Here we care about occupancy on sites. 
2. A broad G-SELEX peak suggests that, for pure DNA binding with a specific protein, the protein can bind anywhere within that region of DNA with comparable affinity. - by ChatGPT
3. Maybe I can focus on HIS3. It starts from 885 to 1953, and its length is 1069 bp or 356 nm. This looks good for a simulation. When the 3D dissociation constant is 1 nm^2/us and $D_{1d}$ is 0.1 nm^2/us, covering the whole length takes $0.063$ s or $0.63$ s for pure diffusion. (Compare $\sqrt{2Dt}$ to the total length).

In [7]:
length_HIS3 = (HIS3['end'] - HIS3['start'])/3 # nm

In [8]:
print("The contour length of HIS3 (including upstream and downstream) is %.1f nm"%length_HIS3)
print("For 1d, D=0.1 nm^/us, when 2Dt = L^2, t=%.3f s"%(length_HIS3**2/2/0.1/1e6)) 
print("For 3d, D=1 nm^/us, when 2Dt = L^2, t=%.3f s"%(length_HIS3**2/2/1e6))

The contour length of HIS3 (including upstream and downstream) is 356.3 nm
For 1d, D=0.1 nm^/us, when 2Dt = L^2, t=0.635 s
For 3d, D=1 nm^/us, when 2Dt = L^2, t=0.063 s


In [9]:
# relative to the start of HIS3, resembling sites are
resembSitesPositions = [
    start-HIS3['start'] for start in startSites_doubleCheck \
    if HIS3['start'] < start < HIS3['end']
]
N_protein = int(round(len(resembSitesPositions) / 10))
print(f"{len(resembSitesPositions)} sites found. Thus {N_protein} proteins will be added. \
(see discussion about Coey and Clark G-SELEX below.)")

53 sites found. Thus 5 proteins will be added. (see discussion about Coey and Clark G-SELEX below.)


Previously we used $n_{non}/l \approx 3/5$ nm$^{-1}$, now this setting is kept. Still, we need to subtract trap (targets) from the total number of nonspecific sites. To be clear, hereafter I will use random sites for random sequences and targets for all traps including the consensus sequence and "half" sites.

In [10]:
N_random = int(round(length_HIS3*3/5 - len(resembSitesPositions),0))
print(f"{N_random} random sites for protein slidnig.")

161 random sites for protein slidnig.


Coey and Clark's G-SELEX uses DNA ~ 15 uM. This is $15\times 6.022 \times 10^{-7}$ segments (~200bp) per $nm^3$, or $1.81 \times 10^{-3}$ bp / nm$^3$. The HIS3 segment has 1069 bp so the volume should be $5.92 \times 10^5$ nm$^3$. The length is already decided to be 356.3 nm, and thus the y, z length should be 40.75 nm $\approx$ 40 nm.

Since there will be only 5 proteins, simply randomly initialize them with multiple trajectories might be the best way for a homogeneous distribution.

In [ ]:
from initiatePositions import write_fixedCrds
InitialCoords = {}
length = length_HIS3
xcrds = np.linspace(-length/2+0.001, length/2-0.001, N_random)
InitialCoords['N'] = [[x-length/2] for x in xcrds]
for i, x in enumerate(resembSitesPositions):
    InitialCoords[f'S{i}'] = [[x-length/2+4.5/3]] # plus 4.5 bp to use the center as position
y_center = 0
z_center = 0
fixCrds = write_fixedCrds(InitialCoords, y_center, z_center)
# print(fixCrds)
with open('./model/fixCoordinates.pdb', 'w') as f:
    f.write(fixCrds)

For equilibrium comparison, kinetic rates do not affect the results. Thus, we can set relative fast kinetic rates (especially off rates). The off rates for target binding can be in the scale of $1 s^{-1}$ to reduce time correlation. 

Should the off rate of random binding regulate the width of G_SELEX peaks?
- Maybe this should be a combination of target-binding off rate and random-binding off rate.
- Wrong! Since, except for the trap position, $\partial \rho/\partial t = D \partial^2 \rho/\partial x^2$, and the reflective boundary condition where $\partial \rho/\partial x = 0$, the only stationary solution is $\rho = const.$ Therefore, the target-binding off rate is not the problem.
- Considering non-equilibrium problem, the distribution of the bound protein along DNA depends on its initial position, and the dynamical distribution follows Smoluchovski's theory. Now, the random-binding off rate affects the measured peak width when its faster than relaxation time for spatial redistribution ($k_a^2 t/D$).
- What about the on rate of target binding? In principal, it can regulate the peak width since it tunes the distribution of the arrival position. However, according to facilitated diffusion's studies, the target binding is at diffusion limit and that is why 1D diffusion can accelerate searching. Therefore, we may assume the initial distribution of proteins along the DNA is uniform.
- For Coey and Clark's work, DNA is in excess (~ 10 sites per protein monomer). 3 rounds of G-SELEX were performed, and they did not mention changing protein-DNA ratio. ref: https://doi.org/10.1101/gr.276080.121
    - They also use gel assay to determine the KD for the native (strongest) binding (with so called RY motifs) is 80 nM.

Summarize:
1. Target-binding rates can be chosen quite freely, just to make sure $k^S_a >> 4\pi D\sigma = 12.566$ nm$^2$/us.
    - As a result of ***diffusion-limit***, we simply assume kaS are the same
2. Random-binding rates might need to be tuned for a better result.
3. Full site affinity is KD = 80 nM and random binding affinity is KD = 80 uM (adopt the empirical difference between dimer specific binding and random binding.)
4. To decide the KD for different traps, this formula is considered:

$$K_D = \exp \frac{\Delta G}{k_BT}$$


In [12]:
def write_PS_reaction(S_name, KD, ka):
    """
    KD in nM
    ka in nm^2 / us
    """
    KD_nm3 = KD * 1e-9 * 0.6022 # convert to nm^-3
    kb = ka * KD_nm3 * 1e6 # this is s^-1
    reaction_str = f"""  {S_name}(bs) + P(dbs) <-> {S_name}(bs!1).P(dbs!1)
    onRate3Dka = {ka:.4f}
    offRatekb = {kb:.4f} # KD = {KD:.2f} nM
    norm1 = [0, 0, 1]
    norm2 = [0, 0, 1]
    sigma = 1.00
    assocAngles = [0.785398, 2.356194, 0.000000, M_PI, 0.000000]
    loopcoopfactor = 1
    bindRadSameCom = 1.1
    area3dto1d = 10
    """
    return reaction_str

In [ ]:
KD_full = 80 # nM
KD_random = 80 * 1e3 # nM

kaS = 120
kaN = 120 # I am looking for kbN ~ 100 kbS
kbN = KD_random * 1e-9 * 0.6022 * kaN * 1e6 

reactions = f"""start reactions

  N(bs) + P(dbn) <-> N(bs!1).P(dbn!1)
    onRate3Dka = {kaN:.4f}
    offRatekb = {kbN:.4f} # KD = {KD_random:.2f} nM
    norm1 = [0, 0, 1]
    norm2 = [0, 0, 1]
    sigma = 1.000007
    assocAngles = [0.785398, 2.356194, 0.000000, M_PI, 0.000000]
    loopcoopfactor = 1
    bindRadSameCom = 1.1
    area3dto1d = 10
    
"""

for si, sx in enumerate(resembSitesPositions):
    KD_si = KD_full ** (9 / matches_doubleCheck[sx+HIS3['start']])
    reactions += write_PS_reaction(f"S{si}", KD_si, kaS)
    reactions += "\n"
reactions += "end reactions"

In [15]:
parameters = f"""# Info:
#    GCN4 binding with HIS3 including upstream and downstream
#    Model G-SELEX measurement by Coey and Clark, 2022
#    For the purpose of Mini-chromatin of Zelin Wei
#    2025-09-16

start parameters
    nItr = 1000000001 # 1e9 iterations
    timeStep = 1 # us, the maximum timestep calculated from kbPN
    timeWrite = 1e5 # 0.1 s
    pdbWrite = 1e5
    trajWrite = 1e9
    assocdissocwrite = false
    overlapSepLimit = 1.0
end parameters

start boundaries
    WaterBox = [356.3, 40, 40] # VtoL = 5.7 x 10^5 nm^3
end boundaries

start molecules
    N : {N_random}
    P : 5
"""

for si, _ in enumerate(resembSitesPositions):
    parameters += f"    S{si} : 1"
    parameters += "\n"
parameters += "end molecules\n\n"
parameters += reactions

with open('./model/parms.inp', 'w') as f:
    f.write(parameters)

# print(parameters)

Then just create molecule files for simulations, P and N can be the same as before, S should be modified by names.

In [16]:
def write_S_mol(S_name):

    molinfo = f"""##
# specific binding sites molecule information file
##

Name    = {S_name}
isPromoter = true

# translational diffusion constants
D       = [0.0,0.0,0.0] # for chromatin

# rotational diffusion constants
Dr      = [0.000,0.000,0.000]

# Coordinates, with states below, or
COM     0.0000    0.0000    0.0000
bs      0.3000   -0.4000    0.5000
re1     0.1000    0.0000    0.0000 # reference points for S+S
re2    -0.1000    0.0000    0.0000 """

    with open(f'./model/{S_name}.mol', 'w') as f:
        f.write(molinfo)
    

In [17]:
for si, _ in enumerate(resembSitesPositions):
    write_S_mol(f"S{si}")

In [18]:
print("To achieve D_3d = 1 and D_1d = 0.1, D_N = %.3f"%(1/9))
print("Actual D_1d:", (1 + 1/0.111)**-1)

To achieve D_3d = 1 and D_1d = 0.1, D_N = 0.111
Actual D_1d: 0.0999099909990999
